In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os

from keras.layers import Concatenate, Input, Dense, Embedding, Flatten, Dropout, BatchNormalization, SpatialDropout1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import  Adam
import keras.backend as k

# import matplotlib.pyplot as plt
# %matplotlib inline
# from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
# from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
# pd.options.display.precision = 15
from category_encoders.cat_boost import CatBoostEncoder

# import lightgbm as lgb
# import xgboost as xgb
# import time
# import datetime
# from catboost import CatBoostRegressor
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
# from sklearn import metrics
# from sklearn import linear_model
import gc
import pickle
# import seaborn as sns
# import warnings
# warnings.filterwarnings("ignore")

# import eli5
# import shap
# from IPython.display import HTML
# import json
# import altair as alt

# import networkx as nx
# import matplotlib.pyplot as plt

gc.collect()
# alt.renderers.enable('notebook')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


0

In [5]:
main_path = r'f:\my\Prog\kaggle\Baydin'

In [6]:
import sys
sys.path.append(main_path)
from BayDS import *

In [7]:
experiment_name = '31.08'
main_learning_folder = main_path+'/Snapshots/'+experiment_name


In [9]:
#start here
data_folder = main_path+'/Data'
y = pd.read_pickle(f'{data_folder}/y.pkl')
X = pd.read_pickle(f'{data_folder}/X_encoded_scaled.pkl').astype(np.float32)
test = pd.read_pickle(f'{data_folder}/test_encoded_scaled.pkl').astype(np.float32)
gc.collect()

15

In [10]:
# Setting model_folder
model_name = 'keras-4'
model_folder = f'{main_learning_folder}/{model_name}'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [11]:
n_fold = 5
# folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=n_fold)
# folds = GroupKFold(n_splits=5)
# groups = pd.read_pickle('./groups.pkl').values

In [12]:
y.value_counts()

0    569877
1     20663
Name: isFraud, dtype: int64

In [13]:
def NNModel_maker():
    k.clear_session()
    
#     categorical_inputs = []
#     for cat in categorical:
#         categorical_inputs.append(Input(shape=[1], name=cat))

#     categorical_embeddings = []
#     for i, cat in enumerate(categorical):
#         categorical_embeddings.append(
#             Embedding(category_counts[cat], int(np.log1p(category_counts[cat]) + 1), name = cat + \
#                       "_embed")(categorical_inputs[i]))

#     categorical_logits = Concatenate(name = "categorical_conc")([Flatten()(SpatialDropout1D(.1)(cat_emb)) for cat_emb in categorical_embeddings])
# 
    numerical_inputs = Input(shape=[X.shape[1]], name = 'all')
    numerical_logits = Dropout(.3)(numerical_inputs)
  
    x = numerical_logits

    x = Dense(400, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = Dense(200, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = BatchNormalization()(x)    
    
    out = Dense(1, activation = 'sigmoid')(x)    

    model = Model(inputs= [numerical_inputs],outputs=out)
    loss = "binary_crossentropy"
    model.compile(optimizer=Adam(lr = 0.0003), loss = loss)
    return model


params = {
    'batch_size': 8192,
    'epochs': 60,
    'verbose': True,
         }
train_options = {
    "model_type":'keras',
    "params": params,
    "eval_metric":'auc',
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
   
}

In [14]:
with open(f'{model_folder}/training_params.json', 'w') as f:
    q = json.dumps(train_options,indent=2)
    f.write(q)


In [15]:
NNModel_maker().save(f'{model_folder}/keras.mdl')

W0906 20:37:32.378583 13944 deprecation_wrapper.py:119] From c:\python36\lib\site-packages\keras\backend\tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0906 20:37:32.379570 13944 deprecation_wrapper.py:119] From c:\python36\lib\site-packages\keras\backend\tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0906 20:37:32.432199 13944 deprecation_wrapper.py:119] From c:\python36\lib\site-packages\keras\backend\tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0906 20:37:32.433181 13944 deprecation_wrapper.py:119] From c:\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0906 20:37:32.441186 13944 deprecation.py:506] From c:\python36\lib\site-packag

In [20]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2422684412784766773, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6696213545
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10341058718385918652
 physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:07:00.0, compute capability: 6.1"]

In [19]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [21]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [22]:
import keras
import tensorflow as tf

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU':8} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [ ]:
gc.collect()
result_dict_keras = train_model_classification(model=NNModel_maker, 
                                             X=X,
                                             X_test=test,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=True,
                                             averaging=train_options['averaging'],
                                             groups=None)

Fold 1 started at Fri Sep  6 21:23:07 2019
Train on 472432 samples, validate on 118108 samples
Epoch 1/60
472432/472432 [==============================] - 21s 45us/step - loss: 0.6592 - val_loss: 0.6178
Epoch 2/60
472432/472432 [==============================] - 20s 42us/step - loss: 0.4463 - val_loss: 0.4971
Epoch 3/60
472432/472432 [==============================] - 22s 46us/step - loss: 0.3049 - val_loss: 0.3780
Epoch 4/60
472432/472432 [==============================] - 21s 45us/step - loss: 0.2237 - val_loss: 0.2995
Epoch 5/60
472432/472432 [==============================] - 21s 45us/step - loss: 0.1780 - val_loss: 0.2453
Epoch 6/60
472432/472432 [==============================] - 21s 45us/step - loss: 0.1515 - val_loss: 0.1896
Epoch 7/60
472432/472432 [==============================] - 21s 45us/step - loss: 0.1354 - val_loss: 0.1763
Epoch 8/60
472432/472432 [==============================] - 21s 45us/step - loss: 0.1259 - val_loss: 0.1446
Epoch 9/60
472432/472432 [===============

Epoch 14/60
472432/472432 [==============================] - 22s 47us/step - loss: 0.0935 - val_loss: 0.0964
Epoch 15/60
472432/472432 [==============================] - 22s 47us/step - loss: 0.0909 - val_loss: 0.0952
Epoch 16/60
472432/472432 [==============================] - 22s 47us/step - loss: 0.0887 - val_loss: 0.0940
Epoch 17/60
472432/472432 [==============================] - 22s 46us/step - loss: 0.0867 - val_loss: 0.0898
Epoch 18/60
472432/472432 [==============================] - 21s 45us/step - loss: 0.0854 - val_loss: 0.0885
Epoch 19/60
472432/472432 [==============================] - 22s 46us/step - loss: 0.0836 - val_loss: 0.0864
Epoch 20/60
472432/472432 [==============================] - 22s 46us/step - loss: 0.0828 - val_loss: 0.0871
Epoch 21/60
472432/472432 [==============================] - 22s 47us/step - loss: 0.0811 - val_loss: 0.0858
Epoch 22/60
472432/472432 [==============================] - 22s 47us/step - loss: 0.0801 - val_loss: 0.0863
Epoch 23/60
472432/

472432/472432 [==============================] - 17s 35us/step - loss: 0.0772 - val_loss: 0.0810
Epoch 28/60
472432/472432 [==============================] - 18s 37us/step - loss: 0.0765 - val_loss: 0.0801
Epoch 29/60
472432/472432 [==============================] - 17s 37us/step - loss: 0.0756 - val_loss: 0.0797
Epoch 30/60
472432/472432 [==============================] - 17s 37us/step - loss: 0.0754 - val_loss: 0.0797
Epoch 31/60
472432/472432 [==============================] - 18s 38us/step - loss: 0.0741 - val_loss: 0.0787
Epoch 32/60
472432/472432 [==============================] - 18s 37us/step - loss: 0.0744 - val_loss: 0.0784
Epoch 33/60
472432/472432 [==============================] - 17s 36us/step - loss: 0.0731 - val_loss: 0.0778
Epoch 34/60
472432/472432 [==============================] - 17s 36us/step - loss: 0.0729 - val_loss: 0.0782
Epoch 35/60
472432/472432 [==============================] - 17s 36us/step - loss: 0.0724 - val_loss: 0.0776
Epoch 36/60
472432/472432 [====

472432/472432 [==============================] - 16s 34us/step - loss: 0.0712 - val_loss: 0.0792
Epoch 41/60
472432/472432 [==============================] - 18s 37us/step - loss: 0.0703 - val_loss: 0.0798
Epoch 42/60
472432/472432 [==============================] - 15s 33us/step - loss: 0.0707 - val_loss: 0.0788
Epoch 43/60
472432/472432 [==============================] - 16s 34us/step - loss: 0.0700 - val_loss: 0.0782
Epoch 44/60
472432/472432 [==============================] - 16s 33us/step - loss: 0.0696 - val_loss: 0.0783
Epoch 45/60
472432/472432 [==============================] - 18s 38us/step - loss: 0.0689 - val_loss: 0.0781
Epoch 46/60
472432/472432 [==============================] - 113s 240us/step - loss: 0.0686 - val_loss: 0.0771
Epoch 47/60
472432/472432 [==============================] - 196s 415us/step - loss: 0.0681 - val_loss: 0.0767
Epoch 48/60
472432/472432 [==============================] - 203s 430us/step - loss: 0.0681 - val_loss: 0.0776
Epoch 49/60
472432/472432

In [ ]:
sub = pd.read_csv(f'../../data/sample_submission.csv')

In [ ]:
sub['isFraud'] = result_dict_keras['prediction']
sub.to_csv(f'{model_folder}/ieee_nn.csv', index=False)

In [ ]:
result_dict_keras['prediction'].shape

In [ ]:
test.shape

In [ ]:
import pickle
with open(f'{model_folder}/results_dict.pkl', 'wb') as f:
#     q = json.dumps(result_dict_lgb,indent=2)
    pickle.dump(result_dict_keras,f)
#     f.write(q)